<a href="https://colab.research.google.com/github/WilderJoseth/Coursera_Capstone/blob/master/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project
In this work I will use borough of Toronto.

In [18]:
!pip install geocoder

     |████████████████████████████████| 102kB 2.2MB/s 


In [19]:
import pandas as pd
import numpy as np
import geocoder # import geocoder

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


###1. Data
This data comes from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M saving it in a CSV file.

In [12]:
ds = pd.read_csv('sample_data/torontoCodPostalList.csv')
ds.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
ds = ds[ds['Borough'] != 'Not assigned']
ds.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
ds.shape

(103, 3)

### 2. Get geographical coordinates

In [22]:
dsGeo = pd.read_csv('http://cocl.us/Geospatial_data')
dsGeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
ds = ds.join(dsGeo.set_index('Postal Code'), on = 'Postal Code')
ds.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
